# **Imports**

In [1]:
import pandas as pd
from dataclasses import dataclass
from openpyxl import load_workbook
import json
pd.set_option('display.max_columns', None)

# **Constants**

In [2]:
@dataclass(frozen=True)
class Pathes:
    # moec_connection_scheme: str = '../main_datasets/processed_datasets/7. Схема подключений МОЭК.csv'
    odpu_heating_uploading: str = '../main_datasets/processed_datasets/11. Выгрузка_ОДПУ_отопление_ВАО__all_in_one.csv'
    apartments_technical_economic_characteristics: str = '../main_datasets/processed_datasets/14. ВАО_Многоквартирные_дома_с_технико_экономическими_характеристиками__ВАО.csv'
    asupr_and_ods_connections: str = '../main_datasets/processed_datasets/8. Данные АСУПР с диспетчерскими ОДС____Связь ЦТП-Потребитель-ОДС.csv'
    bti_uploads: str = '../main_datasets/processed_datasets/9. Выгрузка БТИ.csv'
    adresses_geodata_of_buildings: str = '../main_datasets/processed_datasets/13. Адресный реестр объектов недвижимости города Москвы____Processed.csv'



In [3]:
columns_norm_forms = {
    'УНОМ': 'UNOM',
    'ЦТП': 'Номер ТП',
    'Учётный номер объекта адресации в БД БТИ (кроме помещения и комнаты)': 'UNOM'
}

In [4]:
with open('../columns_info.json', 'r') as f:
    c = json.load(f)['useless_columns']

In [5]:
c

['Кадастровый номер земельного участка (для ОКС)',
 'Код КЛАДР для адресообразующего элемента нижнего уровня',
 'Номер документа о регистрации адреса',
 'ID УУ',
 'ID ТУ',
 'Округ',
 'Серия/Номер счетчика',
 'Unit',
 'Адрес из сторонней системы',
 'Адрес',
 'Город',
 'Общая площадь нежилых помещений',
 'Идентификатор из сторонней системы',
 '№ п/п',
 'global_id',
 'UNAD',
 'Уникальный номер адреса в Адресном реестре']

In [6]:
useless_columns = [
    'Дата', 'Месяц/Год', 'Ошибки', 'Объём поданого теплоносителя в систему ЦО', 
    'Объём обратного теплоносителя из системы ЦО', 'Разница между подачей и обраткой(Подмес)',
    'Разница между подачей и обраткой(Утечка)', 'Температура подачи', 'Температура обратки',
    'Наработка часов счётчика', 'Расход тепловой энергии', 'Ошибки', 'Общая площадь нежилых помещений',
    'Упрощённое написание адреса или описание местоположения', 
]

# **Functions**

In [7]:
def column_preprocessor(columns: pd.Index):
    return pd.Index([columns_norm_forms[i] if i in columns_norm_forms else i for i in columns])


def replace_bad_spaces(df: pd.Index) -> pd.DataFrame:
    c = df.map(lambda a: str(a).rstrip().lstrip())
    return c


def remove_useless_columns(df: pd.DataFrame):
    df = df.loc[:, ~df.columns.isin(useless_columns)]
    return df

In [8]:
def data_frame_loader(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    df.columns = column_preprocessor(df.columns)
    df.columns = replace_bad_spaces(df.columns)
    df = remove_useless_columns(df)
    return df

# **Show of tables**

In [9]:
pd.read_csv(Pathes.adresses_geodata_of_buildings).head()

/tmp/ipykernel_631930/2135451956.py:1: DtypeWarning: Columns (11,13,18,21,34) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(Pathes.adresses_geodata_of_buildings).head()


,global_id,Тип объекта адресации,На территории Москвы,Полное юридическое написание адреса или описание местоположения,Учётный номер объекта адресации в БД БТИ (кроме помещения и комнаты),Субъект РФ,Поселение,Город,Муниципальный округ,Населённый пункт,Наименование элемента планировочной структуры или улично-дорожной сети,Дополнительный адресообразующий элемент,Упрощённое написание адреса или описание местоположения,Уточнение дополнительного адресообразующего элемента,"Тип номера дома, владения, участка","Номер дома, владения, участка",Тип номера корпуса,Страна,Номер корпуса,"Тип номера строения, сооружения",Внутригородская территория,"Номер строения, сооружения",Тип номера помещения,Номер помещения,Тип номера комнаты,Номер комнаты,Административный округ,"Муниципальный округ, поселение",Уникальный номер адреса в Адресном реестре,Дата регистрации адреса в Адресном реестре,Уникальный номер адреса в государственном адресном реестре,Дата регистрации адреса в государственном адресном реестре,Кадастровый номер объекта недвижимости (кроме земельного участка),Кадастровый номер земельного участка (для ОКС),Код КЛАДР для адресообразующего элемента нижнего уровня,Документ-основание регистрационных действий,Номер документа о регистрации адреса,Дата документа о регистрации адреса,Тип адреса,Вид адреса,Состояние адреса,Статус адреса,geoData,geodata_center
0,645622141,Здание,да,"Российская Федерация, город Москва, внутригоро...",2102436,город Москва,NaN,NaN,муниципальный округ Вешняки,NaN,Косинская улица,NaN,"Косинская улица, дом 26А",NaN,дом,26А,NaN,Российская Федерация,NaN,NaN,внутригородская территория муниципальный округ...,NaN,NaN,NaN,NaN,NaN,Восточный административный округ,муниципальный округ Вешняки,3303053.0,03.08.2004,235212A3-01E8-4CC3-87D5-59F00C83898A,27.02.2012,KAD_N:77:03:0007004:1064\n\n,KAD_ZU:77:03:0007004:6443\n\n,77000000000040000,Распоряжение префектуры АО города Москвы,875-В-РП,28.07.2004,Официальный,Адрес,Зарегистрирован в АР,Внесён в ГКН,"{coordinates=[[[37.827950455, 55.717660993], [...","{coordinates=[37.828189394, 55.717482785], typ..."
1,645622142,Здание,да,"Российская Федерация, город Москва, внутригоро...",5783,город Москва,NaN,NaN,муниципальный округ Басманный,NaN,Гороховский переулок,NaN,"Гороховский переулок, дом 21",NaN,дом,21,NaN,Российская Федерация,NaN,NaN,внутригородская территория муниципальный округ...,NaN,NaN,NaN,NaN,NaN,Центральный административный округ,муниципальный округ Басманный,1011856.0,13.07.2005,533D296D-1ECC-49EA-9156-DCFA8E38E4D8,27.02.2012,KAD_N:77:01:0003010:1018\n\n,KAD_ZU:77:01:0003010:4146\n\n,77000000000112200,Распоряжение префектуры АО города Москвы,3026-р,04.07.2005,Официальный,Адрес,Зарегистрирован в АР,Внесён в ГКН,"{coordinates=[[[37.668299521, 55.766243148], [...","{coordinates=[37.668013072, 55.766228272], typ..."
2,645622174,Здание,да,"Российская Федерация, город Москва, внутригоро...",2133081,город Москва,NaN,NaN,муниципальный округ Сокол,NaN,Ленинградский проспект,NaN,"Ленинградский проспект, дом 73А, строение 3",NaN,дом,73А,NaN,Российская Федерация,NaN,строение,внутригородская территория муниципальный округ...,3,NaN,NaN,NaN,NaN,Северный административный округ,муниципальный округ Сокол,9004656.0,05.11.2003,2F3D29A6-D703-45DD-A46D-877D4B2EF7F5,24.11.2011,KAD_N:77:09:0005002:1090\n\n,KAD_ZU:77:09:0005002:45\n\n,77000000000168500,Распоряжение префектуры АО города Москвы,9169,31.10.2003,Официальный,Адрес,Зарегистрирован в АР,Внесён в ГКН,"{coordinates=[[[37.513646239, 55.803629636], [...","{coordinates=[37.513482336, 55.803579031], typ..."
3,645622205,Здание,да,"Российская Федерация, город Москва, внутригоро...",2128644,город Москва,NaN,NaN,муниципальный округ Рязанский,NaN,улица Вострухина,NaN,"улица Вострухина, дом 5А",NaN,дом,5А,NaN,Российская Федерация,NaN,NaN,внутригородская территория муниципальный округ...,NaN,NaN,NaN,NaN,NaN,Юго-Восточный административный округ,муниципальный округ Рязанский,4003036.0,05.01.2004,AE616C0D-D032-47FC-B74E-196CC94D4D53,27.02.2012,KAD_N:77:04:0

In [16]:
pd.read_csv(Pathes.moec_connection_scheme).head()

AttributeError: type object 'Pathes' has no attribute 'moec_connection_scheme'

In [ ]:
pd.read_csv(Pathes.bti_uploads).head()

,№ п/п,Город,Административный округ,Муниципальный округ,Населенный пункт,Улица,Тип номера дом,Номер дома,Номер корпуса,Тип номера строения/сооружения,Номер строения,UNOM,UNAD,Материал,Назначение,Класс,Тип,Этажность,Признак,Общая площадь
0,15.0,город Москва,Восточный административный округ,муниципальный округ Косино-Ухтомский,NaN,Златоустовская улица,дом,21,NaN,NaN,NaN,8450.0,1.0,NaN,NaN,жилые,Здание,NaN,NaN,NaN
1,17.0,город Москва,Восточный административный округ,муниципальный округ Косино-Ухтомский,NaN,Большая Косинская улица,дом,34В,NaN,NaN,NaN,51067.0,1.0,деревянные,индивидуальное жилое строение,жилые,Здание,1.0,NaN,"90,1"
2,18.0,город Москва,Восточный административный округ,муниципальный округ Ивановское,NaN,Зелёный проспект,дом,101,NaN,NaN,NaN,8425.0,1.0,NaN,NaN,жилые,Здание,NaN,NaN,NaN
3,19.0,город Москва,Восточный административный округ,муниципальный округ Ивановское,NaN,Зелёный проспект,дом,103,NaN,NaN,NaN,8426.0,1.0,NaN,NaN,жилые,Здание,NaN,NaN,NaN
4,53.0,город Москва,Восточный административный округ,муниципальный округ Косино-Ухтомский,NaN,улица Чёрное Озеро,дом,7,1,NaN,NaN,28680.0,1.0,NaN,NaN,жилые,Здание,NaN,NaN,NaN


In [ ]:
pd.read_csv(Pathes.apartments_technical_economic_characteristics).head()

,Адрес из сторонней системы,Идентификатор из сторонней системы,Адрес по БТИ,УНОМ,Округ,Район,Серии проектов,Количество этажей,Количество подъездов,Количество квартир,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Износ объекта (по БТИ),Материалы стен,Признак аварийности здания,Количество пассажирских лифтов,Количество грузопассажирских лифтов,Очередность уборки кровли,Материалы кровли по БТИ,Типы жилищного фонда,Статусы МКД
0,"Дом по адресу РФ, г. Москва, вн.тер.г. м.о. Но...",1520521,"город Москва, Суздальская улица, дом 24, корпус 2",31783,Восточный административный округ,муниципальный округ Новокосино,П-55,14,3.0,130.0,9210.9,8615.3,595.6,NaN,панельные,нет,6.0,0.0,NaN,прочая(черепица;щепа;дранка),МКД,в эксплуатации
1,"Дом по адресу РФ, г. Москва, вн.тер.г. м.о. Со...",1526496,"город Москва, улица Сокольнический Вал, дом 4",24231,Восточный административный округ,муниципальный округ Сокольники,нет данных,5,3.0,61.0,2806.9,2603.3,203.6,NaN,из железобетонных сегментов,нет,0.0,0.0,1 очередь,мягкая-совмещенная с рубероидным покрытием,МКД,в эксплуатации
2,"Дом по адресу РФ, г. Москва, вн.тер.г. м.о. Но...",1502303,"город Москва, Новокосинская улица, дом 38, кор...",16560,Восточный административный округ,муниципальный округ Новокосино,П-44,17,2.0,128.0,9538.4,6990.9,2547.5,NaN,панельные,нет,2.0,2.0,NaN,мягкая-совмещенная с рубероидным покрытием,МКД,в эксплуатации
3,"Дом по адресу РФ, г. Москва, вн.тер.г. м.о. Ив...",1502774,"город Москва, Малый Купавенский проезд, дом 1",12180,Восточный административный округ,муниципальный округ Ивановское,П-68,16,2.0,206.0,11502.2,10480.4,1021.8,NaN,железобетонные,нет,2.0,2.0,NaN,мягкая-совмещенная с рубероидным покрытием,МКД,в эксплуатации
4,"Дом по адресу РФ, г. Москва, вн.тер.г. м.о. Из...",1502277,"город Москва, 5-я Парковая улица, дом 52",18617,Восточный административный округ,муниципальный округ Измайлово,нет данных,5,9.0,155.0,9140.0,6932.0,2208,NaN,кирпичные,нет,0.0,0.0,1 очередь,асбофанера-шифер,МКД,в эксплуатации


In [ ]:
pd.read_csv(Pathes.asupr_and_ods_connections).head()

,ID УУ,Адрес,Полный адрес,Округ,UNOM,Группа,№ ОДС,Адрес ОДС,Потребитель (или УК),ЦТП
0,785480981,"10-я Парковая ул., д. 15","город Москва, 10-я Парковая улица, дом 15",ВАО,18777,МКД,ОДС №27-Измайлово,"город Москва, 7-я Парковая улица, дом 15, корп...","ГБУ ""ЖИЛИЩНИК РАЙОНА ИЗМАЙЛОВО""",04-05-0604/185
1,785481133,"10-я Парковая ул., д. 17","город Москва, 10-я Парковая улица, дом 17",ВАО,18778,МКД,ОДС №27-Измайлово,"город Москва, 7-я Парковая улица, дом 15, корп...","ГБУ ""ЖИЛИЩНИК РАЙОНА ИЗМАЙЛОВО""",04-05-0604/185
2,1881452,"10-я Парковая ул., д. 18А","город Москва, 10-я Парковая улица, дом 18А",ВАО,2105327,Бюджет,NaN,NaN,ГКУ ДИРЕКЦИЯ ДОНМ,04-05-0604/124
3,785481250,"10-я Парковая ул., д. 4","город Москва, 10-я Парковая улица, дом 4",ВАО,18773,МКД,ОДС №57-Измайлово,"город Москва, 9-я Парковая улица, дом 27","ГБУ ""ЖИЛИЩНИК РАЙОНА ИЗМАЙЛОВО""",04-05-0604/064
4,785481200,"10-я Парковая ул., д. 6","город Москва, 10-я Парковая улица, дом 6",ВАО,18775,МКД,ОДС №57-Измайлово,"город Москва, 9-я Парковая улица, дом 27","ГБУ ""ЖИЛИЩНИК РАЙОНА ИЗМАЙЛОВО""",04-05-0604/064


# Join by UNOM

In [10]:
dfs = [data_frame_loader(Pathes.__getattribute__(Pathes, i)) for i in list(Pathes.__annotations__.keys())]

/tmp/ipykernel_631930/1423988873.py:2: DtypeWarning: Columns (11,13,18,21,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [17]:
import numpy as np

In [22]:
np.unique(sum([i['UNOM'].unique().tolist() for i in dfs], [])).shape

(491401,)

In [25]:
merged_df = pd.DataFrame({'UNOM': np.unique(sum([i['UNOM'].unique().tolist() for i in dfs], []))})
f = []
for k, i in enumerate(dfs):
    # print(i.shape)
    if 'UNOM' in i.columns:
        print(k)
        f.append(True)
        if merged_df is None:
            merged_df = i
        else:
            merged_df = pd.merge(merged_df, i, on='UNOM', how='left')
    else:
        f.append(False)

0
1
2
3
4


In [28]:
merged_df = merged_df.dropna(axis=1, how='all').drop_duplicates()

In [29]:
c_indexes = []
passed = []
unique_columns = merged_df.columns.map(lambda a: a.replace('_x', '').replace('_y', '')).unique()
for i, c in enumerate(merged_df.columns):
    cc = c.replace('_x', '').replace('_y', '')
    if cc in unique_columns[~unique_columns.isin(passed)]:
        passed.append(cc)
        c_indexes.append(i)


In [33]:
unique_columns

Index(['UNOM', 'ID УУ', 'ID ТУ', 'Округ', 'Район', 'Потребители', 'Группа',
       'Адрес', 'Центральное отопление(контур)', 'Марка счетчика',
       'Серия/Номер счетчика', 'Unit', 'Адрес из сторонней системы',
       'Идентификатор из сторонней системы', 'Адрес по БТИ', 'Серии проектов',
       'Количество этажей', 'Количество подъездов', 'Количество квартир',
       'Общая площадь', 'Общая площадь жилых помещений',
       'Износ объекта (по БТИ)', 'Материалы стен',
       'Признак аварийности здания', 'Количество пассажирских лифтов',
       'Количество грузопассажирских лифтов', 'Очередность уборки кровли',
       'Материалы кровли по БТИ', 'Типы жилищного фонда', 'Статусы МКД',
       'Полный адрес', '№ ОДС', 'Адрес ОДС', 'Потребитель (или УК)',
       'Номер ТП', '№ п/п', 'Город', 'Административный округ',
       'Муниципальный округ', 'Населенный пункт', 'Улица', 'Тип номера дом',
       'Номер дома', 'Номер корпуса', 'Тип номера строения/сооружения',
       'Номер строения', '

In [30]:
clean_df = merged_df.iloc[:, c_indexes]
clean_df.columns = unique_columns

In [34]:
clean_df

,UNOM,ID УУ,ID ТУ,Округ,Район,Потребители,Группа,Адрес,Центральное отопление(контур),Марка счетчика,Серия/Номер счетчика,Unit,Адрес из сторонней системы,Идентификатор из сторонней системы,Адрес по БТИ,Серии проектов,Количество этажей,Количество подъездов,Количество квартир,Общая площадь,Общая площадь жилых помещений,Износ объекта (по БТИ),Материалы стен,Признак аварийности здания,Количество пассажирских лифтов,Количество грузопассажирских лифтов,Очередность уборки кровли,Материалы кровли по БТИ,Типы жилищного фонда,Статусы МКД,Полный адрес,№ ОДС,Адрес ОДС,Потребитель (или УК),Номер ТП,№ п/п,Город,Административный округ,Муниципальный округ,Населенный пункт,Улица,Тип номера дом,Номер дома,Номер корпуса,Тип номера строения/сооружения,Номер строения,UNAD,Материал,Назначение,Класс,Тип,Этажность,global_id,Тип объекта адресации,На территории Москвы,Полное юридическое написание адреса или описание местоположения,Субъект РФ,Поселение,Населённый пункт,Наименование элемента планировочной структуры или улично-дорожной сети,Дополнительный адресообразующий элемент,Уточнение дополнительного адресообразующего элемента,"Тип номера дома, владения, участка","Номер дома, владения, участка",Тип номера корпуса,Страна,"Тип номера строения, сооружения",Внутригородская территория,"Номер строения, сооружения","Муниципальный округ, поселение",Уникальный номер адреса в Адресном реестре,Дата регистрации адреса в Адресном реестре,Уникальный номер адреса в государственном адресном реестре,Дата регистрации адреса в государственном адресном реестре,Кадастровый номер объекта недвижимости (кроме земельного участка),Кадастровый номер земельного участка (для ОКС),Код КЛАДР для адресообразующего элемента нижнего уровня,Документ-основание регистрационных действий,Номер документа о регистрации адреса,Дата документа о регистрации адреса,Тип адреса,Вид адреса,Состояние адреса,Статус адреса,geoData,geodata_center
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,645845415.0,Здание,да,"Российская Федерация, город Москва, внутригоро...",город Москва,NaN,NaN,улица Егора Абакумова,NaN,NaN,дом,3,NaN,Российская Федерация,NaN,внутригородская территория муниципальный округ...,NaN,муниципальный округ Ярославский,2206862.0,07.04.2005,AAD72F1B-106E-4752-A693-75317D0F9A44,27.02.2012,KAD_N:77:02:0013005:1015\n\n,NaN,77000000000070500,Распоряжение префектуры АО города Москвы,674,31.03.2005,Официальный,Адрес,Зарегистрирован в АР,Внесён в ГКН,"{coordinates=[[[37.716405746, 55.878327218], [...","{coordinates=[37.716257791, 55.878117391], typ..."
1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,646721766.0,Здание,да,"Российская Федерация, город Москва, внутригоро...",город Москва,NaN,NaN,Астрадамская улица,NaN,NaN,дом,5,NaN,Российская Федерация,NaN,внутригородская территория муниципальный округ...,NaN,муниципальный округ Тимирязевский,9004773.0,12.11.2003,E11A6A4C-8FA1-43C4-B386-7C2961AE78A2,27.02.2012,KAD_N:77:09:0003021:1025\n\n,NaN,77000000000077400,Распоряжение префектуры АО города Москвы,9231,03.11.2003,Официальный,Адрес,Зарегистрирован в АР,Внесён в ГКН,"{coordinates=[[[37.564970621, 55.811671481], [...","{coordinates=[37.564766438, 55.811763694], typ..."
2,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,647168882.0,Здание,да,"Российская Федерация, город Москва, внутригоро...",город Москва,NaN,NaN,Астрадамская улица,NaN,NaN,дом,6,NaN,Российская Федерация,NaN,внутригородская территория муниципальный округ...,NaN,муниципальный округ Тимирязевский,9008904.0,23.11.2004,636F2F4F-3D65-4293-B774-8872488696DA,27.02.

In [35]:
clean_df.to_csv('../main_datasets/builded_datasets/buildings_description.csv', index=False)